In [1]:
import pandas as pd
import pickle as pkl
import os
import time
import sys
import cx_Oracle

sys.path.append("../")
oracle_conn = cx_Oracle.connect("run", "run", "40.16.17.176:1621/EE.oracle.docker",encoding = "UTF-8",nencoding = "UTF-8")

In [15]:
def translate_opp_account():
    query = "SELECT OPP_ACCT_ID,OPP_CUST_ID,fa.kehuzhao AS ACCT_ID,fe.duifkhzh AS CUST_ID FROM T2A_TRANS t JOIN F_AG_DP_KDPA_ZHXINX fa ON t.ACCT_ID = fa.zhanghao JOIN F_EV_DP_KDPL_ZHMINX fe ON t.ACCT_ID = fe.zhanghao"
    df = pd.read_sql(query,oracle_conn)
    acc_table,cust_table = {},{}
    
#     d = {'OPP_ACCT_ID': [1, 2], 'ACCT_ID': [3, 4]}
#     df = pd.DataFrame(data=d)
#     print(df)

    def func(row):
        acc_table[row["OPP_ACCT_ID"]] = row["ACCT_ID"]
        cust_table[row["OPP_CUST_ID"]] = row["CUST_ID"]

    df.apply(func,axis = 1)
    return acc_table,cust_table

acc_table,cust_table = translate_opp_account()

   OPP_ACCT_ID  ACCT_ID
0            1        3
1            2        4
{1: 3, 2: 4}


In [ ]:
def save_hall():
    query = "SELECT DISTINCT(YNGYJIGO) FROM F_CM_KTLP_GYCSHU"
    df = pd.read_sql(query,oracle_conn)
    df.columns = ["YNGYJIGO:ID"]
    df[":LABEL"] = "Hall"
    print(df.head())
    print(df.shape())
    df._to_csv("data/hall.csv")
    print("save finished")
save_hall()

In [ ]:
def save_hall():
    query = "SELECT DISTINCT(CUST_ID) FROM T2A_TRANS"
    df = pd.read_sql(query,oracle_conn)
    
    
    query_opp = "SELECT DISTINCT(OPP_CUST_ID) as CUST_ID FROM T2A_TRANS"
    df_opp = pd.read_sql(query_opp,oracle_conn)
    
    index = 0
    def func(row):
        df.loc[index,"CUST_ID"] = cust_table[row["CUST_ID"]]
        index += 1
        
    df_opp.apply(func,axis = 1)
    df = df.append(df_opp)
 
    df.columns = ["CUST:ID"]
    df[":LABEL"] = "Customer"
    print(df.head())
    print(df.shape())
    df._to_csv("data/customer.csv")
    print("save finished")
save_hall()

In [ ]:
def save_account():
    query = "SELECT DISTINCT(ACCT_ID) FROM T2A_TRANS"
    df = pd.read_sql(query,oracle_conn)
    
    
    query_opp = "SELECT DISTINCT(OPP_ACCT_ID) as ACCT_ID FROM T2A_TRANS"
    df_opp = pd.read_sql(query_opp,oracle_conn)
    
    index = 0
    def func(row):
        df_opp.loc[index,"ACCT_ID"] = acc_table[row["ACCT_ID"]]
        index += 1
        
    df_opp.apply(func,axis = 1)
    df = df.append(df_opp)
 
    df.columns = ["ACCT:ID"]
    df[":LABEL"] = "ACCT:ID"
    print(df.head())
    print(df.shape())
    df._to_csv("data/account.csv")
    print("save finished")
    
save_account()

In [ ]:
# 不变
def save_teller():
    pass 
    
    

In [ ]:
def save_belongTo_rela():
    query = "SELECT CUST_ID,ACCT_ID FROM T2A_TRANS"
    df = pd.read_sql(query,oracle_conn)
    
    query = "SELECT OPP_CUST_ID AS CUST_ID,OPP_ACCT_ID AS ACCT_ID FROM T2A_TRANS"
    df_opp = pd.read_sql(query,oracle_conn)
    
    index = 0
    def func(row):
        df_opp.loc[index,"ACCT_ID"] = acc_table[row["ACCT_ID"]]
        df_opp.loc[index,"CUST_ID"] = cust_table[row["CUST_ID"]]
        index += 1
    
    df_opp.apply(func,axis = 1)
    df = df.append(df_opp)
    
    df.rename(columns = {"ACCT_ID":":START_ID","CUST_ID":":END_ID"})
    df[":TYPE"] = "belongTo"
    
    print(df.head())
    print(df.shape())
    
    df._to_csv("data/belongTo.csv")
    print("save finished")
    
save_belongTo_rela()

In [ ]:
def save_beOperated_rela():
    query = "SELECT OPR_ID,ACCT_ID FROM T2A_TRANS"
    df = pd.read_sql(query,oracle_conn)
    
    query = "SELECT OPR_ID,OPP_ACCT_ID AS ACCT_ID FROM T2A_TRANS"
    df_opp = pd.read_sql(query,oracle_conn)
    
    index = 0
    def func(row):
        df_opp.loc[index,"ACCT_ID"] = acc_table[row["ACCT_ID"]]
        index += 1
    
    df_opp.apply(func,axis = 1)
    df = df.append(df_opp)
    
    df.rename(columns = {"ACCT_ID":":START_ID","OPR_ID":":END_ID"})
    df[":TYPE"] = "beOperated"
    
    print(df.head())
    print(df.shape())
    df._to_csv("data/beOperated.csv")
    print("save finished")
    
save_beOperated_rela()

In [ ]:
def save_transaction_rela():
    acc_table,cust_table = translate_opp_account()
    query = "SELECT TR_NO,TR_TM,TR_AMT,ACCT_ID,OPP_ACCT_ID FROM T2A_TRANS"
    df = pd.read_sql(query,oracle_conn)
    
    index = 0
    def func(row):
        df.loc[index,"OPP_ACCT_ID"] = acc_table[row["OPP_ACCT_ID"]]
        index += 1
    
    df.apply(func,axis = 1)
    
    df.rename(columns = {"ACCT_ID":":START_ID","OPP_ACCT_ID":":END_ID"}, inplace = True)
    df[":TYPE"] = "transaction"
    df.to_csv("data/transaction.csv")
    print("Save finished")

save_transaction_rela()